In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, Flatten
from sklearn.cluster import KMeans

# Cargar el dataset
data = pd.read_csv("rutas_clustering.csv")

# Aplicar clustering sobre las rutas
num_clusters = 100  # Ajusta según lo que consideres adecuado
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['CLUSTER_ID'] = kmeans.fit_predict(data[['PERMUTACION_ID']])

# Cambiar PERMUTACION_ID por CLUSTER_ID para entrenar el modelo
data['PERMUTACION_ID'] = data['CLUSTER_ID']

# Mapear PERMUTACION_ID a etiquetas consecutivas
permutaciones_unicas = data['PERMUTACION_ID'].unique()
perm_to_index = {perm: idx for idx, perm in enumerate(permutaciones_unicas)}
data['PERMUTACION_ID'] = data['PERMUTACION_ID'].map(perm_to_index)

# Preparar características (X) y etiquetas (y)
X = data.drop(['PERMUTACION_ID'], axis=1).values
y = data['PERMUTACION_ID'].values

# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo
num_classes = len(permutaciones_unicas)

model = Sequential([
    Input(shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Pérdida en validación: {val_loss}, Precisión en validación: {val_accuracy}')

# Guardar el modelo de Keras en formato HDF5
model.save('model.h5')

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0665 - loss: 4.6315 - val_accuracy: 0.1965 - val_loss: 3.3206
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2079 - loss: 3.1218 - val_accuracy: 0.3335 - val_loss: 2.4281
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3565 - loss: 2.2907 - val_accuracy: 0.4700 - val_loss: 1.8673
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5045 - loss: 1.7253 - val_accuracy: 0.5375 - val_loss: 1.5320
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5812 - loss: 1.4485 - val_accuracy: 0.6135 - val_loss: 1.3097
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6196 - loss: 1.2715 - val_accuracy: 0.6105 - val_loss: 1.2533
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6355 - loss: 1.1512 - val_accuracy: 0.6585 - val_loss: 1.1440
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6681 - loss: 1.0695 - val_accuracy: 0.

Pérdida en validación: 0.4829156994819641, Precisión en validación: 0.828499972820282


In [11]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import itertools

# Cargar el modelo entrenado
model = load_model('model.h5')

# Compilar el modelo después de cargarlo para evitar el warning (opcional)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Cargar el archivo con el mapeo entre CLUSTER_ID y PERMUTACION_ID (solo si no lo tienes en el entorno)
# Si ya tienes estos diccionarios, puedes omitir esta parte.
data = pd.read_csv("rutas_clustering.csv")  # Reemplaza con el nombre correcto si es necesario
cluster_to_ruta = data[['CLUSTER_ID', 'PERMUTACION_ID']].drop_duplicates().set_index('CLUSTER_ID')['PERMUTACION_ID'].to_dict()

# Cargar el mapeo de PERMUTACION_ID a rutas de desempeños (id_to_perm)
# Esto se basa en las permutaciones generadas al inicio.
desempenos = [f'D{i+1}' for i in range(10)]
permutaciones = list(itertools.permutations(desempenos))
id_to_perm = {i: perm for i, perm in enumerate(permutaciones)}

# Datos del nuevo estudiante (Ejemplo)
# Ajusta los valores según el estudiante: [SEXO, EDAD, NIVEL_LOGRO, PREG1, PREG2, ..., PREG10]
nuevo_estudiante = np.array([[1, 0.5, 2, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0]])  # Ejemplo con 14 características

# Realizar la predicción
prediccion = model.predict(nuevo_estudiante)

# Obtener el CLUSTER_ID con la mayor probabilidad
cluster_predicho = np.argmax(prediccion)
print(f'CLUSTER_ID predicho para el nuevo estudiante: {cluster_predicho}')

# Obtener el PERMUTACION_ID usando el diccionario `cluster_to_ruta`
perm_id_predicho = cluster_to_ruta.get(cluster_predicho, "PERMUTACIÓN no encontrada")
print(f'PERMUTACION_ID para el CLUSTER_ID {cluster_predicho}: {perm_id_predicho}')

# Obtener la ruta de desempeños completa usando `id_to_perm`
ruta_predicha = id_to_perm.get(perm_id_predicho, "Ruta no encontrada")
print(f'Ruta de desempeños para el PERMUTACION_ID {perm_id_predicho}: {ruta_predicha}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
CLUSTER_ID predicho para el nuevo estudiante: 40
PERMUTACION_ID para el CLUSTER_ID 40: 44
Ruta de desempeños para el PERMUTACION_ID 44: ('D1', 'D2', 'D3', 'D4', 'D5', 'D7', 'D10', 'D8', 'D6', 'D9')
